# Preprocessing

In [11]:
from gensim import corpora
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords

def tokenize(text, stopset):
    tokens = word_tokenize(text)
    return [w for w in tokens if not w in stopset]

df = pd.read_csv("lyrics.csv")
stopset = set(stopwords.words('english')).union(set(['chorus','verse','[',']','(',')']))

df = df[['index','song','artist','genre','lyrics']]
df = df[df.apply(lambda x: not (pd.isnull(x['lyrics']) or pd.isnull(x['song']) or x['genre'] == 'Not Available' or x['genre'] == 'Other'), 
        axis=1, reduce=True)]
df['lyrics'] = df.apply(lambda x: tokenize(x['lyrics'], stopset), axis=1, reduce=True)
df = df[df.apply(lambda x: len(x['lyrics']) > 20, axis=1, reduce=True)]
df = df.groupby('genre').apply(lambda x: x.sample(n=2000))

df.to_pickle('tickle_my.pkl')



# Word/Document Embedding

In [1]:
from gensim import corpora
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


In [2]:
model.init_sims(replace=False)

In [92]:
#create seeds
happy = ['happy']
love = ['love','lust']
sad = ['sad']
hate = ['anger','hate']

#Generate related words from seeds
happy_words50 = [a[0] for a in model.most_similar(positive=sad,negative=happy, topn=50)]
sad_words50 = [a[0] for a in model.most_similar(positive=happy,negative=sad, topn=50)]
love_words50 = [a[0] for a in model.most_similar(positive=love,negative=hate, topn=50)]
hate_words50 = [a[0] for a in model.most_similar(positive=hate,negative=love, topn=50)]


In [94]:
import string

punct = set(string.punctuation)

def doc2vec(model, tokenized_doc):
    count = 0    
    res = np.zeros(300)
    for word in tokenized_doc:
        if word not in punct:
            if word in model:
                count += 1
                res += model[word]
    if count != 0:
        res /= count
    return res

happy_words50_vec = doc2vec(model, happy_words50)
sad_words50_vec = doc2vec(model, sad_words50)
love_words50_vec = doc2vec(model, love_words50)
hate_words50_vec = doc2vec(model, hate_words50)


In [104]:
import pandas as pd

df = pd.read_pickle('tickle_my.pkl')
df['lyrics'] = df.apply(lambda x: doc2vec(model,x['lyrics']), axis=1)


ValueError: Shape of passed values is (20000, 300), indices imply (20000, 5)

genre          
Country  211565    [Who, one-eyed, ?, Aha, !, For, Something, els...
         112059    [An, empty, glass, That, last, cigarette, It, ...
         3166      [Since, day, I, born, mother, used, say, ,, I,...
         159817    [Here, comes, tears, cry, Here, comes, heartac...
         104884    [I, get, purple, heart, loving, pain, put, Whe...
         293837    [Gordon, Kennedy/Wayne, Kirkpatrick/Trisha, Ye...
         91177     [She, locked, fingers, ,, bowed, head, ,, She,...
         129605    [Dolly, Parton/Dorothy, Jo, Hope, In, mental, ...
         298914    [All, dogs, haunted, And, snow, melts, grass, ...
         251500    [I, standing, side, Texas, way, back, home, Me...
         67819     [All, day, I, 've, faced, barren, waste, Witho...
         310132    [Both, Come, baby, let, 's, make, love, war, c...
         129763    [Dolly, Parton, Without, love, call, life, inc...
         129534    [I, 'm, point, n't, add, I, ca, n't, say, I, '...
         270746   